In [154]:
import os, re, logging
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer
import seaborn as sns
import logging
from nltk.corpus import stopwords
from string import punctuation
import nltk
import pandas as pd


BASE_DICT = '/Users/anne/repos/RPA/resources/'
FILENAME_DICT = '20140718_dutchdictionary.txt'
PATH_TO_DATA = '../data/raw/'

MINNUMBERMATCHES = 2 # min number of times a keyword should occur for a topic to be present

stemmer = SnowballStemmer("dutch")

def label_topic(x):
    if x == '1':
        return 'Macro-economie en belastingen'
    if x == '2':
        return 'Burgerrechten en vrijheden'
    if x == '3':
        return 'Gezondheid'
    if x == '4':
        return 'Landbouw en Visserij'
    if x == '5':
        return 'Arbeid'
    if x == '6':
        return 'Onderwijs'
    if x == '7':
        return 'Milieu'
    if x == '8':
        return 'Energiebeleid'
    if x == '9':
        return 'Immigratie en integratie'
    if x == '10':
        return 'Verkeer en vervoer'
    if x == '11':
        return 'Unkown'
    if x == '12':
        return 'Justitie, Rechtspraak, Criminaliteit'
    if x == '13':
        return 'sociale Zaken'
    if x == '14':
        return 'Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning'
    if x == '15':
        return 'Ondernemingen, Bankwezen en binnenlandse handel '
    if x == '16':
        return 'Defensie'
    if x == '17':
        return 'Wetenschappelijk onderzoek, technologie en communicatie'
    if x == '18':
        return 'Buitenlandse handel'
    if x == '19':
        return 'Buitenlandse zaken en ontwikkelingssamenwerking'
    if x == '20':
        return 'Functioneren democratie en openbaar bestuur'
    if x == '21':
        return 'Ruimtelijke ordening, publiek natuur- en waterbeheer'
    if x == '22':
        return 'Unkown 2'
    if x == '23':
        return 'Kunst, cultuur en entertainment'
    if x == '24':
        return '*** Gemeentelijk en provinciaal bestuur'
    if x == '29':
        return '*** Sport'
    if x == '00':
        return 'Toegevoegde codes voor media'

def parse_xml():
    '''reads file with topic numbers + words and parses the title'''

    words = []
    topics = []
    for l in [line.strip() for line in open(os.path.join(BASE_DICT , FILENAME_DICT)).readlines() if len(line)>1] :
        topics_words = defaultdict(list)
        if l.startswith('<cnode'):
            wordlist = []
            topics_l = list(re.sub('">|"|t', '', l.split('=')[1]) )
            if len(topics_l) == 2 :
                final_topic = "".join(topics_l)
            elif len(topics_l) == 3 :
                final_topic = topics_l[0]
            elif len(topics_l) == 4 :
                final_topic = "".join( topics_l[:2] )
        elif l.startswith('<pnode'):
            word = re.sub('">|</pnode>|"', '', l.split('=')[1])
            words.append(word)
            topics.append(final_topic)
    return words, topics

def get_dict():
    'returns a dict with keys = topic, values = words '

    words, topics = parse_xml()
    d = defaultdict(list)
    for topic, word in zip(topics, words):
        topic_name = label_topic(topic)
        d[topic_name].append(word.strip())
    return d

def get_stemmed_dict():
    stemmer = SnowballStemmer("dutch")
    d = get_dict()
    stemmed_dictionary = {}
    for topic, words in d.items():
        stemmed_dictionary[topic] = [ stemmer.stem(w) for w in words ]
    return stemmed_dictionary

def get_raw_data():
    columns_to_keep = ['text', 'topic', 'main_topic_label', 'attrresp', 'attrresp_wrds', 'hmnintrst', 'hmnintrst_wrds', 'cnflct', 'cnflct_wrds', 'ecnmc', 'ecnmc_wrds']

    df = pd.read_pickle(PATH_TO_DATA + 'VK_TEL_merged_with_annotated.pkl')
    df['text_title'] = df['text'].astype(str) + ' ' + df['title'].astype(str)
    del df['text']
    df.rename(columns={'text_title' : 'text', 'main_topic' : 'topic'}, inplace = True)
    df = df[columns_to_keep]
    df['type'] = 'newspaper'

    df2 = pd.read_pickle(PATH_TO_DATA + 'kamervragen_merged_with_annotated')
    df2.rename(columns={'questions' : 'text', 'main_topic' : 'topic'}, inplace = True)
    df2 = df2[columns_to_keep]
    df2['type'] = 'parlementary question'

    df = df.append(df2)

    df['origin'] = 'RPA'
    df.reset_index(drop=True, inplace=True)
    df['documentnr'] = df.index
    logger.info("Appended the kamervragen dataset to the newspaper dataset, resulting in a df with a len of {}".format(len(df)))
    return df

def get_data():
    df = get_raw_data()
    a = ['Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning' , 'Landbouw en Visserij', 'Macro-economie en belastingen', 'Wetenschappelijk onderzoek, technologie en communicatie',  'Toegevoegde codes voor media',  'Buitenlandse handel',  'Kunst, cultuur en entertainment', 'Energiebeleid', 'Ruimtelijke ordening, publiek natuur- en waterbeheer']
    b = ['Overige'] * len(a)
    overige_cat = dict(zip(a,b))
    df['main_topic_label'].replace(overige_cat, inplace = True)
    df.reset_index(drop=True, inplace=True)
    df['documentnr'] = df.index
    logger.info("Retrieved the recoded dataset containing {} cases".format(len(df)))

    body_of_text = df['text'].to_list()
    print(body_of_text[0][:501])

    body_of_text=["".join([l for l in speech if l not in punctuation]) for speech in body_of_text]  #remove punctuation
    body_of_text=[speech.lower() for speech in body_of_text]  # convert to lower case
    body_of_text=[" ".join(speech.split()) for speech in body_of_text]

    mystopwords = stopwords.words('dutch')
    extra_stop = [line.strip() for line in open('../stopwords/stopwords_NL.txt').readlines() if len(line)>1]
    mystopwords = set(mystopwords + extra_stop)
    text_clean = [" ".join([w for w in speech.split() if w not in mystopwords]) for speech in body_of_text]
    text_clean = [" ".join([w for w in speech.split() if w.isalpha()]) for speech in text_clean] # keep only words, no digits
    print("HERE COMES THE CLEANED TEXT: \n\n\n\n")
    print(text_clean[0][:501])
    df['text_clean'] = text_clean
    return df

def stem_sentences(sentence):
    try:
        tokens = sentence.split()
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)
    except:
        return 'NAN'

def return_stemmed_text_columns():
    df = get_data()
    logger.info("Start stemming....")
    df['stemmed_text'] = df.text_clean.apply(stem_sentences)
    return df

def dictionary_topics():

    df1 = return_stemmed_text_columns()
    result = []
    documentnr = -1
    logger.info("Start word search....")

    d = get_dict()

    for document in df['text_clean']:
        documentnr += 1
        topics_per_document = {}


        for topic, words in d.items():

            match = [x for x in words if x in document.lower().split(' ')]


            doc_string = document.lower().split(' ')
            index = [doc_string.index(word) for word in match ]
            try:
                index_smallest = min(index)
            except:
                index_smallest = np.nan

            topics_per_document = {'documentnr' : documentnr,
                                    'topic_label_dictionary': topic,
                                    'index_words' : index,
                                    'smallest_index' : index_smallest,
                                    'len matches' : len(match),
                                    'words matches' : match  ,
                                    'text' : document.lower()}
            result.append(topics_per_document)
    return result

def dictionary_topics_stemmed():
    df1 = return_stemmed_text_columns()
    result = []
    documentnr = -1
    logger.info("Start stemmed word search....")
    
    d = get_stemmed_dict()
    
    for document in df1['stemmed_text']:
        documentnr += 1
        topics_per_document = {}

        for topic, words in d.items():
            match = [x for x in words if x in document.lower().split(' ')]
            doc_string = document.lower().split(' ')
            index = [doc_string.index(word) for word in match ]
            try:
                index_smallest = min(index)
            except:
                index_smallest = np.nan

            topics_per_document = {'documentnr' : documentnr,
                                    'stemmed_topic_label_dictionary': topic,
                                    'stemmed_index_words' : index,
                                    'stemmed_smallest_index' : index_smallest,
                                    'stemmed_len matches' : len(match),
                                    'stemmed_words matches' : match  ,
                                    'stemmed_text' : document.lower()}
            result.append(topics_per_document)
            
    return result

def get_merged_df():
    '''returns a df with number of topics as identified by the dictionary approach'''

    result = dictionary_topics()
    stemmed_results = dictionary_topics_stemmed()
    
    
    df2 = pd.DataFrame.from_dict(result)
    df_mosts = df2.loc[df2.groupby(["documentnr"])["len matches"].idxmax()]  
    df_mosts = df_mosts[['documentnr', 'len matches' , 'words matches' , 'topic_label_dictionary']]
    df_mosts.rename(columns = {'len matches' : 'len-matches-freq', 'words matches' : 'word-matched-freq',  'topic_label_dictionary' : 'topic_label_dictionary-freq' }, inplace=True)


    df2 = (df2.assign(to_sort = df2.smallest_index.abs()).sort_values('to_sort').drop_duplicates('documentnr').drop(columns='to_sort'))
    df2 = df2[np.isfinite(df2['smallest_index'])]
    df2 = pd.merge(df2, df_mosts, on='documentnr')
    
    
    df3 = pd.DataFrame.from_dict(stemmed_results)
    
    df_mosts = df3.loc[df3.groupby(["documentnr"])["stemmed_len matches"].idxmax()]  
    
    df_mosts = df_mosts[['documentnr', 'stemmed_len matches' , 'stemmed_words matches' , 'stemmed_topic_label_dictionary']]
    
    df_mosts.rename(columns = { 'stemmed_len matches' : 'stemmed_len-matches-freq', 'stemmed_words matches' : 'stemmed_words-matches-freq', 
    'stemmed_topic_label_dictionary' : 'stemmed_topic_label_dictionary-freq' }, inplace=True)
    
    df3 = (df3.assign(to_sort = df3.stemmed_smallest_index.abs()).sort_values('to_sort').drop_duplicates('documentnr').drop(columns='to_sort'))
    df3 = df3[np.isfinite(df3['stemmed_smallest_index'])]
    df3 = pd.merge(df3, df_mosts, on='documentnr')
    
    df1 = get_data()
    
    df = pd.merge(df1, df2, how= 'left', on = 'documentnr')
    df = pd.merge(df, df3, how = 'left', on='documentnr')
    df['topic_label_dictionary'].fillna(value='Overige', inplace = True)
    df['len matches'] = df['len matches'].fillna(0)
    df['stemmed_topic_label_dictionary'].fillna(value='Overige', inplace = True)
    df['stemmed_len matches'] = df['stemmed_len matches'].fillna(0)
    return df

def recode_dictionary():
    '''recode categories so to match Bjorns' scoring'''

    df = get_merged_df()
    a = ['Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning' , 'Landbouw en Visserij', 'Macro-economie en belastingen', 'Wetenschappelijk onderzoek, technologie en communicatie', 'Toegevoegde codes voor media', 'Buitenlandse handel', 'Kunst, cultuur en entertainment', 'Energiebeleid', 'Ruimtelijke ordening, publiek natuur- en waterbeheer' ,'*** Sport', '*** Gemeentelijk en provinciaal bestuur', 'Ruimtelijke ordening, publiek natuur- en waterbeheer', 'Toegevoegde codes voor media']
    b = ['Overige' ] * len(a)
    overige_cat = dict(zip(a,b))

    df['main_topic_label'].replace(overige_cat, inplace = True)
    df['topic_label_dictionary'].replace(overige_cat, inplace = True)
    df['stemmed_topic_label_dictionary'].replace(overige_cat, inplace = True)
    df['topic_label_dictionary-freq'].replace(overige_cat, inplace = True)
    df['stemmed_topic_label_dictionary-freq'].replace(overige_cat, inplace = True)    

    logger.info("the length of categories identified by dict is now: {} ".format(len(df['topic_label_dictionary'].unique()) ) )
    logger.info("...and the stemmed dict: {} ".format(len(df['stemmed_topic_label_dictionary'].unique()) ) )
    return df



def apply_minnummatches():

    ''' specify how many words should match before the topic is considered present'''
    df = recode_dictionary()
   
    df['topic_label_dictionary_minmatches'] = np.where(df['len matches'] < MINNUMBERMATCHES, 'Overige', df['topic_label_dictionary'])
    df['topic_label_dictionary_minmatches_stem'] = np.where(df['stemmed_len matches'] < MINNUMBERMATCHES, 'Overige', df['stemmed_topic_label_dictionary'])

    df['topic_label_dictionary_minmatches-freq'] = np.where(df['len-matches-freq'] < MINNUMBERMATCHES, 'Overige', df['topic_label_dictionary'])
    df['topic_label_dictionary_minmatches_stem-freq'] = np.where(df['stemmed_len-matches-freq'] < MINNUMBERMATCHES, 'Overige', df['stemmed_topic_label_dictionary'])

    
    ref_cols = ['text_x', 'main_topic_label', 'hmnintrst_wrds','attrresp', 'attrresp_wrds', 'cnflct', 'cnflct_wrds', 'ecnmc', 'ecnmc_wrds','hmnintrst', 'hmnintrst_wrds']
    df = df.loc[~df[ref_cols].duplicated()]
    df['main_topic_label'] = df['main_topic_label'].replace(np.nan, 'Overige')
    
    df['topic_label_dictionary-freq'].replace(np.nan, 'Overige', inplace=True)
    df['stemmed_topic_label_dictionary-freq'].replace(np.nan, 'Overige', inplace=True)
    
    return df

def get_tp_fp_fn():

    '''create columns with true postives, false positives, and false negatives'''

    df = apply_minnummatches()
    topics = list(df['main_topic_label'].unique())

    for topic in topics:
        columnname_tp = "_tp " + str(topic)
        columnname_fp = "_fp " + str(topic)
        columnname_fn = "_fn " + str(topic)
        columnname_tn = "_tn " + str(topic)
        
        
        columnname_tp_freq = "_tp_freq " + str(topic)
        columnname_fp_freq = "_fp_freq " + str(topic)
        columnname_fn_freq = "_fn_freq " + str(topic)
        columnname_tn_freq = "_tn_freq " + str(topic)

        # and for stemmed

        columnname_tp_st = "st_tp " + str(topic)
        columnname_fp_st = "st_fp " + str(topic)
        columnname_fn_st = "st_fn " + str(topic)
        columnname_tn_st = "st_tn " + str(topic)
        
        
        
        columnname_tp_st_freq = "st_tp_freq " + str(topic)
        columnname_fp_st_freq = "st_fp_freq " + str(topic)
        columnname_fn_st_freq = "st_fn_freq " + str(topic)
        columnname_tn_st_freq = "st_tn_freq " + str(topic)

        # true positives = dictionary correctly identified.
        df[columnname_tp] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches'] == topic) , 1, 0 )
        # false positive = dictionary identified, but golden standard not.
        df[columnname_fp] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches'] == topic) , 1, 0 )
        # false negative = dictionary NOT identified, but golden standard DID identify
        df[columnname_fn] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches'] != topic) , 1, 0 )
        df[columnname_tn] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches'] != topic) , 1, 0 )

        
        df[columnname_tp_freq] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches-freq'] == topic) , 1, 0 )
        # false positive = dictionary identified, but golden standard not.
        df[columnname_fp_freq] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches-freq'] == topic) , 1, 0 )
        # false negative = dictionary NOT identified, but golden standard DID identify
        df[columnname_fn_freq] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches-freq'] != topic) , 1, 0 )
        df[columnname_tn_freq] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches-freq'] != topic) , 1, 0 )

        
        # and for stemmed:
        df[columnname_tp_st] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches_stem'] == topic) , 1, 0 )
        df[columnname_fp_st] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches_stem'] == topic) , 1, 0 )
        df[columnname_fn_st] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches_stem'] != topic) , 1, 0 )
        df[columnname_tn_st] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches_stem'] != topic) , 1, 0 )

        df[columnname_tp_st_freq] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches_stem-freq'] == topic) , 1, 0 )
        df[columnname_fp_st_freq] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches_stem-freq'] == topic) , 1, 0 )
        df[columnname_fn_st_freq] = np.where( (df['main_topic_label'] == topic) & (df['topic_label_dictionary_minmatches_stem-freq'] != topic) , 1, 0 )
        df[columnname_tn_st_freq] = np.where( (df['main_topic_label'] != topic) & (df['topic_label_dictionary_minmatches_stem-freq'] != topic) , 1, 0 )

    return df

def main():
    df = get_tp_fp_fn()
    print("Done! Created a df with {} cases.........".format(len(df)))
    df.to_pickle('{}RPA_data_with_dictionaryscores.pkl'.format(PATH_TO_DATA))

if __name__ == '__main__':
    logger = logging.getLogger()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)

    main()

2021-09-17 14:58:16,585 : INFO : Appended the kamervragen dataset to the newspaper dataset, resulting in a df with a len of 2866
2021-09-17 14:58:16,589 : INFO : Retrieved the recoded dataset containing 2866 cases


 Van onze parlementaire redactie DEN HAAG, zondag Vanwege grote kritiek op een geheel door Taiwan betaalde reis, hebben drie CDA-Kamerleden besloten een deel van de omstreden reis zelf te betalen. Daarover ontstond gisteren beroering toen bleek dat de CDA'ers De Haan, Ormel en Haverkamp morgen op kosten van Taiwan met hun partner naar het land afreizen voor een vijfdaagse reis. De Taiwanese regering zou de reis, het verblijf en de onkosten van de Kamerleden inclusief partner betalen. Vorig jaar r


2021-09-17 14:58:17,264 : INFO : Start stemming....


HERE COMES THE CLEANED TEXT: 




parlementaire redactie den haag zondag grote kritiek geheel taiwan betaalde reis cdakamerleden besloten deel omstreden reis betalen daarover ontstond gisteren beroering bleek cdaers haan ormel haverkamp morgen kosten taiwan partner land afreizen vijfdaagse reis taiwanese regering reis verblijf onkosten kamerleden inclusief partner betalen vorig reisden vvders wilders weekers partners kosten taiwan land jaren daarvoor kamerleden regelmatig land afgereisd reizen landen gemaakt kosten daarvan enkele


2021-09-17 14:58:21,003 : INFO : Start word search....


...

HERE COMES THE CLEANED TEXT: 




parlementaire redactie den haag zondag grote kritiek geheel taiwan betaalde reis cdakamerleden besloten deel omstreden reis betalen daarover ontstond gisteren beroering bleek cdaers haan ormel haverkamp morgen kosten taiwan partner land afreizen vijfdaagse reis taiwanese regering reis verblijf onkosten kamerleden inclusief partner betalen vorig reisden vvders wilders weekers partners kosten taiwan land jaren daarvoor kamerleden regelmatig land afgereisd reizen landen gemaakt kosten daarvan enkele


2021-09-17 14:59:35,478 : INFO : Start stemmed word search....
2021-09-17 15:00:18,516 : INFO : Appended the kamervragen dataset to the newspaper dataset, resulting in a df with a len of 2866
2021-09-17 15:00:18,519 : INFO : Retrieved the recoded dataset containing 2866 cases


 Van onze parlementaire redactie DEN HAAG, zondag Vanwege grote kritiek op een geheel door Taiwan betaalde reis, hebben drie CDA-Kamerleden besloten een deel van de omstreden reis zelf te betalen. Daarover ontstond gisteren beroering toen bleek dat de CDA'ers De Haan, Ormel en Haverkamp morgen op kosten van Taiwan met hun partner naar het land afreizen voor een vijfdaagse reis. De Taiwanese regering zou de reis, het verblijf en de onkosten van de Kamerleden inclusief partner betalen. Vorig jaar r


2021-09-17 15:00:19,203 : INFO : the length of categories identified by dict is now: 14 
2021-09-17 15:00:19,204 : INFO : ...and the stemmed dict: 14 


HERE COMES THE CLEANED TEXT: 




parlementaire redactie den haag zondag grote kritiek geheel taiwan betaalde reis cdakamerleden besloten deel omstreden reis betalen daarover ontstond gisteren beroering bleek cdaers haan ormel haverkamp morgen kosten taiwan partner land afreizen vijfdaagse reis taiwanese regering reis verblijf onkosten kamerleden inclusief partner betalen vorig reisden vvders wilders weekers partners kosten taiwan land jaren daarvoor kamerleden regelmatig land afgereisd reizen landen gemaakt kosten daarvan enkele
Done! Created a df with 2807 cases.........


In [153]:
df = pd.read_pickle('../data/raw/RPA_data_with_dictionaryscores.pkl')
recall_score(df['main_topic_label'], df['topic_label_dictionary'], average='weighted', sample_weight=None)


df['topic_label_dictionary-freq'].replace(np.nan, 'Overige', inplace=True)

In [122]:
list(df)

['text_x',
 'topic',
 'main_topic_label',
 'attrresp',
 'attrresp_wrds',
 'hmnintrst',
 'hmnintrst_wrds',
 'cnflct',
 'cnflct_wrds',
 'ecnmc',
 'ecnmc_wrds',
 'type',
 'origin',
 'documentnr',
 'text_clean',
 'topic_label_dictionary',
 'index_words',
 'smallest_index',
 'len matches',
 'words matches',
 'text_y',
 'len-matches-freq',
 'word-matched-freq',
 'topic_label_dictionary-freq',
 'stemmed_topic_label_dictionary',
 'stemmed_index_words',
 'stemmed_smallest_index',
 'stemmed_len matches',
 'stemmed_words matches',
 'stemmed_text',
 'stemmed_len-matches-freq',
 'stemmed_words-matches-freq',
 'stemmed_topic_label_dictionary-freq',
 'topic_label_dictionary_minmatches',
 'topic_label_dictionary_minmatches_stem',
 'topic_label_dictionary_minmatches-freq',
 'topic_label_dictionary_minmatches_stem-freq',
 '_tp Functioneren democratie en openbaar bestuur',
 '_fp Functioneren democratie en openbaar bestuur',
 '_fn Functioneren democratie en openbaar bestuur',
 '_tn Functioneren democratie

In [155]:

import pandas as pd
import logging
import json
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

PATH_TO_DATA = '../data/intermediate/'
FILENAME = 'RPA_data_with_dictionaryscores.pkl'

OUTPUT_PATH ='../output/'

def get_data():
    df = pd.read_pickle('../data/raw/RPA_data_with_dictionaryscores.pkl')
    
    df['topic_label_dictionary-freq'].replace(np.nan, 'Overige', inplace=True)
    df['stemmed_topic_label_dictionary-freq'].replace(np.nan, 'Overige', inplace=True)
   
    return df

def get_recall_precision(topics):

    true_positives = ["_tp " + str(i) for i in topics]
    false_positives = ["_fp " + str(i) for i in topics]
    false_negatives = ["_fn " + str(i) for i in topics]

    true_positives_freq = ["_tp_freq " + str(i) for i in topics]
    false_positives_freq = ["_fp_freq " + str(i) for i in topics]
    false_negatives_freq = ["_fn_freq " + str(i) for i in topics]
    
    true_positives_st = ["st_tp " + str(i) for i in topics]
    false_positives_st = ["st_fp " + str(i) for i in topics]
    false_negatives_st = ["st_fn " + str(i) for i in topics]
    
    true_positives_st_freq = ["st_tp_freq " + str(i) for i in topics]
    false_positives_st_freq = ["st_fp_freq " + str(i) for i in topics]
    false_negatives_st_freq = ["st_fn_freq " + str(i) for i in topics]


    recall = {}
    precision = {}
    f1score = {}
    accuracy = {}

    recall_freq = {}
    precision_freq = {}
    f1score_freq = {}
    accuracy_freq = {}

    recall_stemmed = {}
    precision_stemmed = {}
    f1score_stemmed = {}
    accuracy_stemmed = {}
    
    recall_stemmed_freq = {}
    precision_stemmed_freq = {}
    f1score_stemmed_freq = {}
    accuracy_stemmed_freq = {}

    df = get_data()


    for tp, fp, fn, st_tp, st_fp, st_fn, topic in zip(true_positives, false_positives, false_negatives, true_positives_st, false_positives_st, false_negatives_st, topics) :

        recall[str(topic)] = df[tp].sum(axis=0) / ( df[tp].sum(axis=0) + df[fn].sum(axis=0) )
        precision[str(topic)] = df[tp].sum(axis=0) / ( df[tp].sum(axis=0) + df[fp].sum(axis=0) )
        f1score[str(topic)] = 2 * ( ( precision[str(topic)] * recall[str(topic)] ) / ( precision[str(topic)] + recall[str(topic)] ) )
        accuracy[str(topic)] = 'NaN'

        recall_stemmed[str(topic)] = df[st_tp].sum(axis=0) / ( df[st_tp].sum(axis=0) + df[st_fn].sum(axis=0) )
        precision_stemmed[str(topic)] = df[st_tp].sum(axis=0) / ( df[st_tp].sum(axis=0) + df[st_fp].sum(axis=0) )
        f1score_stemmed[str(topic)] = 2 * ( ( precision_stemmed[str(topic)] * recall_stemmed[str(topic)] ) / ( precision_stemmed[str(topic)] + recall_stemmed[str(topic)] ) )
        accuracy_stemmed[str(topic)] = 'NaN'

    print("LENGTH OF THE DF\n\n")
    print(len(df))

    recall['Accuracy'] = recall_score(df['main_topic_label'], df['topic_label_dictionary'], average='weighted', sample_weight=None)
    precision['Accuracy'] = precision_score(df['main_topic_label'], df['topic_label_dictionary'], average='weighted', sample_weight=None)
    f1score['Accuracy'] = f1_score(df['main_topic_label'], df['topic_label_dictionary'], average='weighted', sample_weight=None)

    recall_stemmed['Accuracy'] = recall_score(df['main_topic_label'], df['stemmed_topic_label_dictionary'], average='weighted', sample_weight=None)
    precision_stemmed['Accuracy'] = precision_score(df['main_topic_label'], df['stemmed_topic_label_dictionary'], average='weighted', sample_weight=None)
    f1score_stemmed['Accuracy'] = f1_score(df['main_topic_label'], df['stemmed_topic_label_dictionary'], average='weighted', sample_weight=None)

    
    
    for tp, fp, fn, st_tp, st_fp, st_fn, topic in zip(true_positives_freq, false_positives_freq, false_negatives_freq, true_positives_st_freq, false_positives_st_freq, false_negatives_st_freq, topics) :

        recall_freq[str(topic)] = df[tp].sum(axis=0) / ( df[tp].sum(axis=0) + df[fn].sum(axis=0) )
        precision_freq[str(topic)] = df[tp].sum(axis=0) / ( df[tp].sum(axis=0) + df[fp].sum(axis=0) )
        f1score_freq[str(topic)] = 2 * ( ( precision[str(topic)] * recall[str(topic)] ) / ( precision[str(topic)] + recall[str(topic)] ) )
        accuracy_freq[str(topic)] = 'NaN'

        recall_stemmed_freq[str(topic)] = df[st_tp].sum(axis=0) / ( df[st_tp].sum(axis=0) + df[st_fn].sum(axis=0) )
        precision_stemmed_freq[str(topic)] = df[st_tp].sum(axis=0) / ( df[st_tp].sum(axis=0) + df[st_fp].sum(axis=0) )
        f1score_stemmed_freq[str(topic)] = 2 * ( ( precision_stemmed[str(topic)] * recall_stemmed[str(topic)] ) / ( precision_stemmed[str(topic)] + recall_stemmed[str(topic)] ) )
        accuracy_stemmed_freq[str(topic)] = 'NaN'

  #  print("LENGTH OF THE DF\n\n")
  #  print(len(df))

    recall_freq['Accuracy'] = recall_score(df['main_topic_label'], df['topic_label_dictionary-freq'], average='weighted', sample_weight=None)
    precision_freq['Accuracy'] = precision_score(df['main_topic_label'], df['topic_label_dictionary-freq'], average='weighted', sample_weight=None)
    f1score_freq['Accuracy'] = f1_score(df['main_topic_label'], df['topic_label_dictionary-freq'], average='weighted', sample_weight=None)

    recall_stemmed_freq['Accuracy'] = recall_score(df['main_topic_label'], df['stemmed_topic_label_dictionary-freq'], average='weighted', sample_weight=None)
    precision_stemmed_freq['Accuracy'] = precision_score(df['main_topic_label'], df['stemmed_topic_label_dictionary-freq'], average='weighted', sample_weight=None)
    f1score_stemmed_freq['Accuracy'] = f1_score(df['main_topic_label'], df['stemmed_topic_label_dictionary-freq'], average='weighted', sample_weight=None)

    
    return recall, precision, f1score, recall_stemmed, precision_stemmed, f1score_stemmed, recall_freq, precision_freq, f1score_freq, recall_stemmed_freq, precision_stemmed_freq, f1score_stemmed_freq



def get_scores():
    df = get_data()
    topics = list(df['main_topic_label'].unique())

    recall, precision, f1score, recall_stemmed, precision_stemmed, f1score_stemmed, recall_freq, precision_freq, f1score_freq, recall_stemmed_freq, precision_stemmed_freq, f1score_stemmed_freq = get_recall_precision(topics)


    total = { k: [ precision[k] , recall[k], f1score[k]] for k in recall }  
    total_freq = { k: [ precision_freq[k] , recall_freq[k], f1score_freq[k]] for k in recall_freq }
    
    total_stemmed =  { k: [ precision_stemmed[k],  recall_stemmed[k] , f1score_stemmed[k]  ] for k in recall_stemmed }

    fname = '{}precision_recall_f1score_dictionary_not_stemmed.json'.format(OUTPUT_PATH)

    with open(fname, mode='w') as fo:
        json.dump(total, fo)

    logging.info("Created file {} with precision, recall and f1scores & accuracy".format(fname))

    
    fname = '{}precision_recall_f1score_dictionary_not_stemmed_freq.json'.format(OUTPUT_PATH)
    
    with open(fname, mode='w') as fo:
        json.dump(total_freq, fo)

    logging.info("Created file {} with precision, recall and f1scores & accuracy FREQ".format(fname))
    
    fname_stemmed = '{}precision_recall_f1score_dictionary_stemmed.json'.format(OUTPUT_PATH)

    with open(fname_stemmed, mode='w') as fo:
        json.dump(total_stemmed, fo)

    logging.info("Created file {} with precision, recall and f1scores & accuracy".format(fname_stemmed))

    fname_stemmed = '{}precision_recall_f1score_dictionary_stemmed_freq.json'.format(OUTPUT_PATH)

    with open(fname_stemmed, mode='w') as fo:
        json.dump(total_stemmed_freq, fo)

    logging.info("Created file {} with precision, recall and f1scores & accuracy".format(fname_stemmed))


if __name__ == "__main__":

    logger = logging.getLogger()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)

    get_scores()


In [157]:

df = get_data()
topics = list(df['main_topic_label'].unique())


recall, precision, f1score, recall_stemmed, precision_stemmed, f1score_stemmed, recall_freq, precision_freq, f1score_freq, recall_stemmed_freq, precision_stemmed_freq, f1score_stemmed_freq = get_recall_precision(topics)


total = { k: [ precision[k] , recall[k], f1score[k]] for k in recall }  
total_freq = { k: [ precision_freq[k] , recall_freq[k], f1score_freq[k]] for k in recall_freq }
    
total_stemmed =  { k: [ precision_stemmed[k],  recall_stemmed[k] , f1score_stemmed[k]  ] for k in recall_stemmed }
total_stemmed_freq =  { k: [ precision_stemmed_freq[k],  recall_stemmed_freq[k] , f1score_stemmed_freq[k]  ] for k in recall_stemmed_freq }


LENGTH OF THE DF


2807


2021-09-17 15:20:34,574 : INFO : Created file ../output/precision_recall_f1score_dictionary_not_stemmed.json with precision, recall and f1scores & accuracy
2021-09-17 15:20:34,575 : INFO : Created file ../output/precision_recall_f1score_dictionary_not_stemmed_freq.json with precision, recall and f1scores & accuracy FREQ
2021-09-17 15:20:34,577 : INFO : Created file ../output/precision_recall_f1score_dictionary_stemmed.json with precision, recall and f1scores & accuracy
2021-09-17 15:20:34,579 : INFO : Created file ../output/precision_recall_f1score_dictionary_stemmed_freq.json with precision, recall and f1scores & accuracy


LENGTH OF THE DF


2807
